| InvoiceMonth | TotalSales |
|--------------|------------|
| Jan          | 108.90     |
| Feb          | 82.17      |
| Mar          | 79.20      |
| Apr          | 86.13      |
| May          | 91.08      |
| Jun          | 37.62      |
